In [1]:
using Oscar
using Random
using Test

## Rozwiązywanie układów kongruencji

In [6]:
gcdx(5,7)

a = 5 
X = Matrix{typeof(a)}(undef,3,5)
A = [1,2,3]
length(A)

3

### Metoda Lagrange'a

In [95]:
function congruence_system(m,a)
    k = length(m)     
    δ = fill(zero(m[1]),k,k)
    for i=1:k
        for j=i+1:k
            _, δ[i,j], δ[j,i] = gcdx(m[i],m[j])
        end
    end
    M = prod(m)
    A = []
    for i=1:k
        Δ = prod(δ[j, i] for j in 1:size(δ, 1) if j != i)
        x = div(M,m[i]) 
        push!(A,Δ*x)
    end
    return mod(sum(a .* A),M)  
end

congruence_system (generic function with 1 method)

In [114]:
R,x = QQ[:x]
f = x
g = x -1
h = x -2

x - 2

In [115]:
congruence_system([f,g,h],[0,1,0])

-x^2 + 2*x

In [88]:
m = [5,7,9]
a = [1,6,5]
x = congruence_system(m,a)

41

### Metoda Przyrostowy 
Algorytm pomocniczy

In [110]:
function congruence_two(m,a)
    _, α, β = gcdx(m[1], m[2])
    r = mod(a[1],m[1])
    b = (a[2] - r)*α 
    s = mod(b,m[2])
    return r + s*m[1]
end

congruence_two (generic function with 1 method)

In [112]:
congruence_two([3,5], [0,1])

6

In [134]:
function congruence_system_newton(m,a)
    Ring = parent(m[1])
    a = Ring.(a)
    k = length(m)
    M = Ring(1)
    A = Ring(1)
    for i=1:k
        b = congruence_two([M,m[i]], [A, a[i]])
        A = b 
        M *= m[i] 
    end
    return A 
end

congruence_system_newton (generic function with 1 method)

In [135]:
m = [5,7,9]
a = [1,6,5]
congruence_system_newton(m,a)

41

In [136]:
R,x = QQ[:x]
f = x
g = x - 1
h = x - 2
congruence_system_newton([f,g,h],[0,1,4])

x^2

## Potęgowanie modularne

### Basic

In [ ]:
function power_modulo(a,n,m)
    d = digits(n,base=2)
    k = length(d)
    r = 1
    b = mod(a,m)
    for i=1:k
        if(d[i]==1)
            r = mod(r*b,m)
        end 
        b = mod(b^2,m)
    end
    return r 
end

power_modulo (generic function with 1 method)

In [191]:
power_modulo(49,19,11)

9

### Funkcja $\varphi$ Eulera

In [201]:
function ϕ(n) 
    res = 1
    for (p, α) in factor(n)
        res*=(p^α - p^(α -1))
    end
    return res 
end

ϕ (generic function with 1 method)

In [205]:
ϕ(450)

120

In [208]:
function power_modulo_euler(a,n,m)
    d = gcd(a,m)
    p = 1 
    if(d>1)
        a = div(a,d)
        p = power_modulo(d,n,m)
    end
    n = mod(n, ϕ(m))
    return p*power_modulo(a,n,m)
end

power_modulo_euler (generic function with 1 method)

In [209]:
power_modulo_euler(2,1000,225)

151

# Liczenie wyznacznika Macierzy Całkowitej

## Nierówność Hadamarda

In [261]:
function hadamard_bounding(M)
    entries = collect(M)                    
    m = maximum(abs.(entries))             
    n = size(M, 1)                           
    return ceil(typeof(M[1,1]),m^n * sqrt(n^n))
end

hadamard_bounding (generic function with 1 method)

In [262]:
M  = matrix(ZZ, 3, 3, [3, 1, -2,
                     2, -3, -1,
                     1, 3, 0])

[3    1   -2]
[2   -3   -1]
[1    3    0]

In [263]:
hadamard_bounding(M)

141

In [236]:
F = GF(7)

Prime field of characteristic 7

In [ ]:
function sieve_eratosthenes(n::Int)
    n < 2 && return Int[]
    
    sieve = trues(n)
    sieve[1] = false 
    
    for i in 2:isqrt(n)
        if sieve[i]
            sieve[i^2:i:n] .= false  
        end
    end
    
    return findall(sieve) .|> Int
end

sieve_eratosthenes (generic function with 1 method)

In [255]:
primes = sieve_eratosthenes(500000)

41538-element Vector{Int64}:
      2
      3
      5
      7
     11
     13
     17
     19
     23
     29
      ⋮
 499883
 499897
 499903
 499927
 499943
 499957
 499969
 499973
 499979

In [ ]:
F = GF(7)
M3 = matrix(F, 3, 3, [3, 1, -2,
                     2, -3, -1,
                     1, 3, 0])
#det(M3)

1
curr 1
matrix [3 1 5; 2 4 6; 1 3 0]
2
curr 3
matrix [3 1 5; 2 4 6; 1 3 0]
whaat 5
whaat2 0


0

In [ ]:
M  = matrix(ZZ, 3, 3, [3, 1, -2,
                     2, -3, -1,
                     1, 3, 0])

[3    1   -2]
[2   -3   -1]
[1    3    0]

In [280]:
det(M)

-10

In [273]:
M2 = [3 1 -2;
     2 -3 -1;
    1 3 0]

3×3 Matrix{Int64}:
 3   1  -2
 2  -3  -1
 1   3   0

In [274]:
det(M2)

-9.999999999999998